In [1]:
# import modules
import pandas as pd
import numpy as np
from datetime import datetime
from os import path
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
from tkcalendar import DateEntry
import sys
from tkinter import Tk, Label, Button, Canvas, ttk
from tkinter.ttk import Entry
import tkinter as tk
import tkinter
import tkinter.messagebox
import time
import threading
import csv

#import GeneratorClass module. This is the calculation module for generator data.
# generator list
genNameList = ['1A', '2A', '3A', '4A', '1B', '2B', '3B', '4B']
genFullName = ['GEN ' + i for i in genNameList]

# categories to be saved in the XLSX file
data_dictionary = {'StartDate': [], 'StartTime': [], 'StopDate': [], 'StopTime': [], 'RunHourStart': [], \
                   'RunHourStop': [], 'Load': [], 'ReasonForRun': [], 'RunTime': [], 'CalcFuel': []}
# These variables are going to be used in 'generator' function
capacity, derate, qtrLoad, halfLoad, threeQtrLoad, fullLoad = 0, 0, 0, 0, 0, 0


# function for creating XLSX file in the storage
def mkexcel(genName, dataframe):
    if not path.isfile('Gen_Data.xlsx'):
        wb = openpyxl.Workbook()
        wb1 = wb['Sheet']
        wb1.title = genName
        for r in dataframe_to_rows(dataframe, index=False, header=True):
            wb1.append(r)

        return wb.save(filename='Gen_Data.xlsx')
    else:
        wb = openpyxl.load_workbook(filename='Gen_Data.xlsx')
        try:
            df = pd.read_excel('Gen_Data.xlsx', sheet_name=genName)
            dataframe = df.append(dataframe, ignore_index=False)
            wb1 = wb[genName]
            wb.remove(wb1)
            wb1 = wb.create_sheet(genName)
        except:
            wb1 = wb.create_sheet(genName)
        for r in dataframe_to_rows(dataframe, index=False, header=True):
            wb1.append(r)
        wb.save(filename='Gen_Data.xlsx')
        
        
        return wb.save(filename='Gen_Data.xlsx')


# function for bringing the information of generators
def generator(genName):
    global capacity, derate, qtrLoad, halfLoad, threeQtrLoad, fullLoad, curRunHour
    if genName in genNameList[0:6]:
        capacity, derate, qtrLoad, halfLoad, threeQtrLoad, fullLoad = 2000, 1795, 43.00, 71.00, 103.00, 135.00
    else:
        capacity, derate, qtrLoad, halfLoad, threeQtrLoad, fullLoad = 2000, 1550, 46.50, 82.00, 107.30, 141.30
    # get the cumulative run hour from the last entered data
    try:
        temp_df = pd.read_excel('Gen_Data.xlsx', sheet_name='GEN ' + genName)
        curRunHour = temp_df.iloc[len(temp_df) - 1]['RunHourStop']
    except:
        # print('\nThere is no data for the cumulative run hours.\nPlease enter the cumulative run hours.')
        # curRunHour = input()
        # while not curRunHour.isdigit():
        #     print('\nPlease enter correct number in hour.')
        #     curRunHour = input()
        # curRunHour = make a field for cumulative run hours or a separate diaglog popped up when user enters the submit
        curRunHour = 3
        curRunHour = float(curRunHour)


# function to convert the input date into datetime format, and to save as list form
def inputdate(date_entry):
    try:
        date = datetime.strptime(date_entry, "%m-%d-%y")
        return [date.strftime("%x")]
    except:
        print(r"[Error] Please use valid form in mm-dd-yy")
        inputdate(input())


# function to convert the input time into datetime format, and to save as list form
def inputtime(time_entry):
    try:
        date = datetime.strptime(time_entry, "%H:%M")
        return [date.strftime("%X")[:5]]
    except:
        print(r"[Error] Please use valid form in hh:mm")
        inputtime(input())


# function to calculate the difference between 'start time' and 'stop time'
def run_time(sdate, stime, edate, etime):
    tempdate1 = sdate + " " + stime
    tempdate2 = edate + " " + etime
    print('Date1:',tempdate1)
    print('Date2:',tempdate2)
    diff = datetime.strptime(tempdate2, "%Y-%m-%d %H:%M") - datetime.strptime(tempdate1, "%Y-%m-%d %H:%M")
    days = diff.days
    totalSeconds = diff.seconds
    hours, remainder = divmod(totalSeconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    hours += days * 24
    return ["{:02d}:{:02d}".format(hours, minutes)]


# function to convert %H:%M format into hours
def cum_run_hour(runHour_now, runtime):
    i, j = map(int, runtime[0].split(':'))
    return [round(runHour_now + i + j / 60, 1)]


# function to calculate the amount of fuel consumed based on 'run time'
def calc_fuel(value, operTime):
    if value / derate < 0.25:
        i, j = map(int, operTime.split(":"))
        return [round((i * qtrLoad) + (j * qtrLoad / 60), 2)]
    elif value / derate < 0.5:
        i, j = map(int, operTime.split(":"))
        return [round((i * halfLoad) + (j * halfLoad / 60), 2)]
    elif value / derate < 0.75:
        i, j = map(int, operTime.split(":"))
        return [round((i * threeQtrLoad) + (j * threeQtrLoad / 60), 2)]
    else:
        i, j = map(int, operTime.split(":"))
        return [round((i * fullLoad) + (j * fullLoad / 60), 2)]

def check_capacity(generatorname):
    genName = generatorname.upper()
    generator(genName)
    wholestr = "Capacity: {}\nDerate: {}\n1/4 Load: {}\n1/2 Load: {}\n3/4 Load: {}\nFull Load: {}" \
        .format(capacity, derate, qtrLoad, halfLoad, threeQtrLoad, fullLoad)
    return wholestr


def dump_data(genname, startdate, starttime, enddate, endtime, loadvalue, reasonRun):
    global curRunHour
    
    data_dictionary['StartDate'] = startdate
    data_dictionary['StartTime'] = starttime
    data_dictionary['StopDate'] = enddate
    data_dictionary['StopTime'] = endtime

    print('SDate',startdate)
    print('STime', starttime)
    print('Edate', enddate)
    print('Etime', endtime)

    # sdate, stime, edate, etime = data_dictionary['StartDate'][0], data_dictionary['StartTime'][0], \
    #                              data_dictionary['StopDate'][0], data_dictionary['StopTime'][0]

    # define dictionary for run times
    data_dictionary['RunTime'] = run_time(startdate, starttime, enddate, endtime)

    # runhourstart
    data_dictionary['RunHourStart'] = [curRunHour]

    # runhourstop
    curRunHour = cum_run_hour(curRunHour, data_dictionary['RunTime'])
    data_dictionary['RunHourStop'] = curRunHour

    data_dictionary['Load'] = [int(loadvalue)]

    data_dictionary['ReasonForRun'] = [reasonRun]

    # calculate fuel
    value = int(data_dictionary['Load'][0])
    operTime = data_dictionary['RunTime'][0]
    data_dictionary['CalcFuel'] = calc_fuel(value, operTime)

    # create excel file
    genName = 'GEN ' + genname
    df = pd.DataFrame(data_dictionary)
    df['StartDate'] = pd.to_datetime(df['StartDate'])
    df['StartDate'] = df['StartDate'].dt.strftime('%Y-%m-%d')
    df['StopDate'] = pd.to_datetime(df['StopDate'])
    df['StopDate'] = df['StopDate'].dt.strftime('%Y-%m-%d')    
    
    mkexcel(genName, df)
    secondresult = 'The excel file has been created.'
    return secondresult

def edit_row(genName, index, startDate, startTime, stopDate, stopTime, loadValue, runReason):
    generator(genName)
    global curRunHour
  
    data_dictionary['StartDate'] = startDate
    data_dictionary['StartTime'] = startTime
    data_dictionary['StopDate'] = stopDate
    data_dictionary['StopTime'] = stopTime
    
    
    # data_dictionary['RunTime'] = runtime(sdate, stime, edate, etime)
    data_dictionary['RunTime'] = run_time(startDate, startTime, stopDate, stopTime)

    # runhourstart
    data_dictionary['RunHourStart'] = [curRunHour]

    # runhourstop
    data_dictionary['RunHourStop'] = cum_run_hour(curRunHour, data_dictionary['RunTime'])
    
    data_dictionary['Load'] = [int(loadValue)]

    data_dictionary['ReasonForRun'] = [runReason]

    # calculate fuel
    data_dictionary['CalcFuel'] = calc_fuel(int(data_dictionary['Load'][0]), data_dictionary['RunTime'][0])

    # create excel file
    fullName = 'GEN ' + genName
    
    dataframe = pd.DataFrame(data_dictionary)
    
    
    delete_data(genName, index)
    
    for i in [0,1]:
        wb = openpyxl.load_workbook(filename='Gen_Data.xlsx')
        wb1 = wb[fullName]
        wb.remove(wb1)
        wb1 = wb.create_sheet(fullName)
        if i == 0:
            df = pd.read_excel('Gen_Data.xlsx', sheet_name=fullName)
            df = df.append(dataframe, ignore_index=False)
        else:
            df = pd.read_excel('Gen_Data.xlsx', sheet_name=fullName, parse_dates=['StartDate','StopDate'])
            origin = df['RunHourStart'][0]
            # sort the data by date
            df = df.sort_values(['StartDate','StartTime'])
            df['StartDate'] = df['StartDate'].dt.strftime('%Y-%m-%d')
            df['StopDate'] = df['StopDate'].dt.strftime('%Y-%m-%d')
            # re-calculate cumRunHours
            df['RunHourStart'] = df['RunHourStart'].astype(float)
            df['RunHourStart'][0] = origin
            for x in range(len(df)):
                i, j = map(int, df['RunTime'][x].split(':'))
                df['RunHourStop'][x] = round(df['RunHourStart'][x] + i + j / 60,1 )
                df['RunHourStart'][x+1] = df['RunHourStop'][x]
            
        for r in dataframe_to_rows(df, index=False, header=True):
            wb1.append(r)        
        wb.save(filename='Gen_Data.xlsx')
    
    
    #wb = openpyxl.load_workbook(filename='Gen_Data.xlsx')
    #sheet = wb[fullName]
    #row = sheet[index+2]
    #for cell in row:
    #    c=0
    #    cell.value = df.iloc[0, c]
    #    c+=1
      

    secondresult = 'The data file has been updated.'
    return secondresult

def delete_data(genName, index):
    fullName = ('GEN ' +genName)
    
    wb = openpyxl.load_workbook(filename='Gen_Data.xlsx')
    sheet = wb[fullName]
    row = index+2
    sheet.delete_rows(index+2)
    wb.save(filename='Gen_Data.xlsx')
    
# function to filter by date and to sum of the sorted values
def sum_hours(genName, month, year):
    ans = '00:00'
    # filter by year
    tempDF = df_dict[genName][pd.DatetimeIndex(df_dict[genName]['StartDate']).year == year]
    # fliter by month
    tempDF = tempDF[pd.DatetimeIndex(tempDF['StartDate']).month == month]
    tempDF = tempDF.reset_index(drop=True)
    if len(tempDF.index) == 1:
        return tempDF['RunTime'][0]
    elif len(tempDF.index) > 1:
        temp = tempDF['RunTime'].str.split(':')
        hourVal, minuteVal = 0, 0
        for i in temp:
            hourVal += int(i[0])
            minuteVal += int(i[1])
        h , m = divmod(minuteVal,60)
        hourVal += h
        ans = "{:02d}:{:02d}".format(hourVal, m)
        return ans
    else:
        return ans

# function to calculate the total hours
def total_hour(genName):
    totHour = 0
    totMinute = 0
    for i in df_MonthlyHour[genName]:
        totHour += int(i.split(':')[0])
        totMinute += int(i.split(':')[1])
    totHour += totMinute / 60
    return totHour

df_dict = {}

def create_summary(year):
    year = int(year)
    # import all generator data
    for gen in genFullName:
        try:
            df_dict[gen] = pd.read_excel('Gen_Data.xlsx', sheet_name=gen, parse_dates=['StartDate', 'StopDate'])
        except:
            pass
        # create dictionary for monthly run hours
        monthlyRunHour = {}

        # calculate the sum of run hours per month
        for gen in genFullName:
            monthlySum = []
            try:
                for mon in range(1, 13):
                    try:
                        monthlySum.append(sum_hours(gen, mon, year))
                    except:
                        monthlySum.append('01:00')
            except:
                monthlySum = ['00:00', '00:00', '00:00', '00:00', '00:00', '00:00', '00:00', '00:00', '00:00', '00:00',
                              '00:00',
                              '00:00']
            monthlyRunHour[gen] = monthlySum

        # create data frame for monthly run hours

        global df_MonthlyHour
        df_MonthlyHour = pd.DataFrame(monthlyRunHour,
                                      index=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                                             'September', 'October', 'November', 'December'])
        # create dictionary for totals
        totals = {}

        # calculate the total values
        for gen in genFullName:
            totalList = []
            # total run hours
            totalList.append(total_hour(gen))
            # total fuel usage
            try:
                temp_fuel = df_dict[gen][pd.DatetimeIndex(df_dict[gen]['StartDate']).year == year]
                totalList.append(int(sum(temp_fuel['CalcFuel'])))
            except:
                totalList.append(0)
            
            # total NOx
            if gen in ['GEN 4A', 'GEN 4B']:
                totalList.append(totalList[0] * 34.11 / 2000)
            else:
                totalList.append(totalList[0] * 35.43 / 2000)
            # total SO2
            if gen in ['GEN 4A', 'GEN 4B']:
                totalList.append(totalList[0] * 0.71 / 2000)
            else:
                totalList.append(totalList[0] * 0.97 / 2000)
            # total Co
            if gen in ['GEN 4A', 'GEN 4B']:
                totalList.append(totalList[0] * 1.16 / 2000)
            else:
                totalList.append(totalList[0] * 2.25 / 2000)
            # total PM 10
            if gen in ['GEN 4A', 'GEN 4B']:
                totalList.append(totalList[0] * 0.26 / 2000)
            else:
                totalList.append(totalList[0] * 0.64 / 2000)
            # total VOC
            if gen in ['GEN 4A', 'GEN 4B']:
                totalList.append(totalList[0] * 0.2 / 2000)
            else:
                totalList.append(totalList[0] * 0.4 / 2000)

            # add to dictionary
            totals[gen] = totalList

        # create data frame for the total values
        df_Totals = pd.DataFrame(totals,
                                 index=['Total Run Hours', 'Total Fuel Usage', 'Total NOx', 'Total SO2', 'Total CO',
                                        'Total PM 10', 'Total VOC']).round(2)

        # join two data frames
        df_summary = pd.concat([df_MonthlyHour, df_Totals])

        # annual limit for emissions
        annual_limit = {'AIRS_Point': ['001', '002', 'Total Point'], 'PM10': [], 'NOx': [], 'VOC': [], 'CO': [], \
                        'Emission_Type': ['Point', 'Point', '']}
        df_air1 = df_summary[['GEN 1A', 'GEN 2A', 'GEN 3A', 'GEN 1B', 'GEN 2B', 'GEN 3B']]
        df_air2 = df_summary[['GEN 4A', 'GEN 4B']]
        annual_limit['PM10'].append(round(sum(df_air1.loc['Total PM 10']), 2))
        annual_limit['PM10'].append(round(sum(df_air2.loc['Total PM 10']), 2))
        annual_limit['PM10'].append(round(sum(annual_limit['PM10']), 2))
        annual_limit['NOx'].append(round(sum(df_air1.loc['Total NOx']), 2))
        annual_limit['NOx'].append(round(sum(df_air2.loc['Total NOx']), 2))
        annual_limit['NOx'].append(round(sum(annual_limit['NOx']), 2))
        annual_limit['VOC'].append(round(sum(df_air1.loc['Total VOC']), 2))
        annual_limit['VOC'].append(round(sum(df_air2.loc['Total VOC']), 2))
        annual_limit['VOC'].append(round(sum(annual_limit['VOC']), 2))
        annual_limit['CO'].append(round(sum(df_air1.loc['Total CO']), 2))
        annual_limit['CO'].append(round(sum(df_air2.loc['Total CO']), 2))
        annual_limit['CO'].append(round(sum(annual_limit['CO']), 2))
        df_annual_limit = pd.DataFrame(annual_limit)

        # annual limit for process
        process_limit = {'AIRS_Point': ['001', '002'],
                         'Process Parameter': ['Hours of Operation per Engine', 'Hours of Operation per Engine'], \
                         'Annual RunHour(hr/yr)': []}
        process_limit['Annual RunHour(hr/yr)'].append(round(max(df_air1.loc['Total Run Hours']), 2))
        process_limit['Annual RunHour(hr/yr)'].append(round(max(df_air2.loc['Total Run Hours']), 2))
        df_process_limit = pd.DataFrame(process_limit)

        # check if value does not pass the annual limits
        if (df_annual_limit.loc[0, 'NOx'] > 25.98) or (df_annual_limit.loc[1, 'NOx'] > 9.69) \
                or (df_annual_limit.loc[0, 'CO'] > 6.17) or (df_annual_limit.loc[1, 'CO'] > 0.93):
            report_1 = "[Warning] There is excess of the annual emission limit."
        else:
            report_1 = "[Safe] Annual emission test passed."
        # check if value does not pass the process limits
        if (df_process_limit.loc[0, 'Annual RunHour(hr/yr)'] > 250) or (
                df_process_limit.loc[1, 'Annual RunHour(hr/yr)'] > 250):
            report_2 = "[Warning] There is excess of the annual process/consumption limit."
        else:
            report_2 = "[Safe] Annual process/consumption test passed."
        # export data frame as excel

        wb = openpyxl.Workbook()
        # sheet for summary table
        wb1 = wb['Sheet']
        wb1.title = 'Summary{}'.format(year)
        for r in dataframe_to_rows(df_summary, index=True, header=True):
            if r == [None]:
                pass
            else:
                wb1.append(r)
        # sheet for annual limit table
        wb2 = wb.create_sheet("Annual_Limit")
        for r in dataframe_to_rows(df_annual_limit, index=False, header=True):
            if r == [None]:
                pass
            else:
                wb2.append(r)
        wb2.append([None])
        for r in dataframe_to_rows(df_process_limit, index=False, header=True):
            if r == [None]:
                pass
            else:
                wb2.append(r)
        wb2.append([None])
        wb2.append(['Emission Test', report_1])
        wb2.append(['RunHour Test', report_2])
        wb.save(filename="Summary_{}.xlsx".format(year))
        print("\nThe 'Summary_{}.xlsx' file has been created.".format(year))
        sumaryfile = "The Summary.xlsx file has been created"
    return sumaryfile

def start():
    global progressbar
    progressbar  = ttk.Progressbar(window, orient = 'horizontal', length = 300, mode = 'determinate')
    canvas1.create_window(400, 220, window=progressbar)
    progressbar.start()

# tkinter decision box function
def yes_no(currWin):
    root = currWin
    canvas1 = tk.Canvas(root, width = 300, height = 300)
    canvas1.pack()
    answer = False
    
    MsgBox = tk.messagebox.askquestion ('Yes/No','Are you sure you want to do this?',icon = 'warning')
    if MsgBox == 'yes':
        answer = True
    return answer   

# login function to validate username and password combination and set admin level
def login():
    userName = str(userNameEntry.get())
    password = str(passwordEntry.get())
    (name, pw, admin) = check_login(userName, password)
    if name == True and pw == True:
        loginWin.withdraw()
        userNameEntry.delete(0, 'end')
        passwordEntry.delete(0, 'end')
        global adminRights
        if admin == True:
            adminRights = True
        select_operation(loginWin)
    else:
        if name == True and pw == False:
            tkinter.messagebox.showerror('Entry Error', "Invalid Password")
            passwordEntry.delete(0, 'end')
        else:
            tkinter.messagebox.showerror('Entry Error', "Invalid Username")
            userNameEntry.delete(0, 'end')
            passwordEntry.delete(0, 'end')

# verify login credentials based on credential file
def check_login(uname, pword):
    name = False
    pw = False
    rights = False
    #load data from credentials file into dataframe
    loginData = pd.read_excel('credentials.xlsx', sheet_name='creds')
    loginData = loginData.set_index('username')
    #check if user name is valid
    if uname in loginData.index:
        name = True
        #pull password for entered username
        actPass = loginData.loc[uname, 'password']
        #verify password is correct for username
        if pword == actPass:
            pw = True
            # set rights level to be returned if password and username combination is valid
            if loginData.loc[uname, 'authlvl'] == 'admin':
                rights = True    
    return (name, pw, rights)

#signout function - closes window and returns to login screen.
def signout(leaveWin):
    #close last window
    leaveWin.destroy()
    #disable admin rights
    global adminRights 
    adminRights = False
    #reopen login window
    loginWin.update()
    loginWin.deiconify()

#function to close window
def close_prev(prevWin):
    prevWin.destroy()

#function for determining which user window to show
def select_operation(prevWin):
    #close the previous window if it was not the login window
    if not prevWin == loginWin:
        close_prev(prevWin)
    #set select window based on admin rights
    global adminRights
    if adminRights == True:
        admin_window()
    else:
        basic_window()

def admin_window():
    adminWindow = tk.Toplevel(loginWin)
    adminWindow.title("Generator Data Interface")
    canvas2 = Canvas(adminWindow, width=720, height=250, relief='raised')
    canvas2.pack()
    enterDataBtn = Button(adminWindow,text='Enter the Generator Data', bg='green', fg='white',command=lambda : enter_data(adminWindow),
                          font=('helvetica', 14))
    canvas2.create_window(340, 50, window=enterDataBtn)
    viewdatabtn = Button(adminWindow,text='View/Edit Generator Data', bg='green', fg='white',command=lambda : view_window(adminWindow),
                              font=('helvetica', 14))
    canvas2.create_window(340, 100, window=viewdatabtn)
    runreportbtn = Button(adminWindow,text='Run Annual Report', bg='green', fg='white',command=lambda : report_gen_win(adminWindow),
                              font=('helvetica', 14))
    canvas2.create_window(340, 150, window=runreportbtn)
    logoutBtn = Button(adminWindow,text='Logout', bg='green', fg='white', command=lambda :signout(adminWindow),
                              font=('helvetica', 14))
    canvas2.create_window(340, 200, window=logoutBtn)
    
def basic_window():
    newWindow = tk.Toplevel(loginWin)
    newWindow.title("Generator Data Interface")
    canvas2 = Canvas(newWindow, width=720, height=300, relief='raised')
    canvas2.pack()
    viewdatabtn = Button(newWindow,text='View Generator Data', bg='green', fg='white',command=lambda : view_window(newWindow),
                              font=('helvetica', 14))
    canvas2.create_window(340, 100, window=viewdatabtn)
    runreportbtn = Button(newWindow,text='Run Annual Report', bg='green', fg='white',command=lambda : report_gen_win(newWindow),
                              font=('helvetica', 14))
    canvas2.create_window(340, 150, window=runreportbtn)
    logoutBtn = Button(newWindow,text='Logout', bg='green', fg='white', command=lambda :signout(newWindow),
                              font=('helvetica', 14))
    canvas2.create_window(340, 200, window=logoutBtn)

def view_window(prevWin):
    close_prev(prevWin)
    newWindow2 = tk.Toplevel(loginWin)
    newWindow2.title("View Generator Data")
    canvas3 = Canvas(newWindow2, width=820, height=300, relief='raised')
    canvas3.pack()

    genNameLabel = Label(newWindow2, text='Please select the generator from the following:\n' + ', '.join(genNameList))
    genNameLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 50, window=genNameLabel)

    global genNameEntry
    genNameEntry = Entry(newWindow2)
    canvas3.create_window(510, 50, window=genNameEntry)

    startDateLabel = Label(newWindow2,
                         text='Enter the start date in format mm-dd-yy')
    startDateLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 90, window=startDateLabel)
    
    global startDateEntry
    startDateEntry = DateEntry(newWindow2)
    canvas3.create_window(510, 90, window=startDateEntry)

    stopDateLabel = Label(newWindow2,
                           text='Enter the stop date in format mm-dd-yy')
    stopDateLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 120, window=stopDateLabel)

    global stopDateEntry
    stopDateEntry = DateEntry(newWindow2)
    canvas3.create_window(510, 120, window=stopDateEntry)

    submitDataBtn = Button(newWindow2, text='View Data', bg='green', fg='white', command=displaydata,
                          font=('helvetica', 14))
    canvas3.create_window(440, 160, window=submitDataBtn)
    global adminRights
    if adminRights == True:
        editdatabtn = Button(newWindow2,text='Edit Data', bg='green', fg='white',command=lambda : edit_data(newWindow2),
                                  font=('helvetica', 14))
        canvas3.create_window(440, 210, window=editdatabtn)
    mainMenuBtn = Button(newWindow2, text='Main Menu', bg='green', fg='white', command=lambda : select_operation(newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(440, 260, window=mainMenuBtn)
        
def enter_data(prevWin):
    close_prev(prevWin)
    newWindow2 = tk.Toplevel(loginWin)
    newWindow2.title("Enter Generator Data")
    canvas3 = Canvas(newWindow2, width=820, height=500, relief='raised')
    canvas3.pack()

    genNameLabel = Label(newWindow2, text='Please select the generator from the following:\n' + ', '.join(genNameList))
    genNameLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 50, window=genNameLabel)

    global genNameEntry
    genNameEntry = Entry(newWindow2)
    canvas3.create_window(510, 50, window=genNameEntry)

    startDateLabel = Label(newWindow2,
                         text='Enter the start date in format mm-dd-yy')
    startDateLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 90, window=startDateLabel)

    global startDateEntry
    startDateEntry = DateEntry(newWindow2)
    canvas3.create_window(510, 90, window=startDateEntry)

    startTimeLabel = Label(newWindow2,
                           text='Enter the start time in format hh:mm')
    startTimeLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 130, window=startTimeLabel)

    global startTimeEntry
    startTimeEntry = Entry(newWindow2)
    canvas3.create_window(510, 130, window=startTimeEntry)
    # timeentry = time.strftime('%H:%M%p')
    #
    # startTimeEntry = Entry(newWindow2, cursor='plus')
    # startTimeEntry.place(x=0, y=0)
    # startTimeEntry.insert(0, timeentry)
    # canvas3.create_window(510, 130, window=startTimeEntry)

    stopDateLabel = Label(newWindow2,
                           text='Enter the stop date in format mm-dd-yy')
    stopDateLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 170, window=stopDateLabel)

    global stopDateEntry

    stopDateEntry = DateEntry(newWindow2)
    canvas3.create_window(510, 170, window=stopDateEntry)

    stopTimeLabel = Label(newWindow2,
                           text='Enter the stop time in format hh:mm')
    stopTimeLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 210, window=stopTimeLabel)

    global stopTimeEntry

    stopTimeEntry = Entry(newWindow2)
    canvas3.create_window(510, 210, window=stopTimeEntry)

    loadValueLabel = Label(newWindow2,
                          text='Enter the value of load')
    loadValueLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 250, window=loadValueLabel)

    global loadValueEntry
    loadValueEntry = Entry(newWindow2)
    canvas3.create_window(510, 250, window=loadValueEntry)

    runReasonLabel = Label(newWindow2,
                          text='Enter the reason for run')

    runReasonLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 290, window=runReasonLabel)

    global runReasonEntry
    runReasonEntry = Entry(newWindow2)
    canvas3.create_window(510, 290, window=runReasonEntry)

    submitDataBtn = Button(newWindow2, text='Insert the Data', bg='green', fg='white', command=submit_data,
                          font=('helvetica', 14))
    canvas3.create_window(440, 350, window=submitDataBtn)
    mainMenuBtn = Button(newWindow2, text='Main Menu', bg='green', fg='white', command=lambda : select_operation(newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(440, 400, window=mainMenuBtn)
    logoutBtn = Button(newWindow2,text='Logout', bg='green', fg='white', command=lambda :signout(newWindow2),
                              font=('helvetica', 14))
    canvas3.create_window(440, 450, window=logoutBtn)


def check_time(starttime,startTimeEntry):
    status = False
    if (len(starttime) >0):
        if(starttime.__contains__(':') and len(starttime)==5):
            first,second = starttime.split(':')
            first = int(first)
            second = int(second)
            if(first>=0 and first<=23):
                if (second >= 0 and second <= 59):
                    status = True
                else:
                    tkinter.messagebox.showinfo("Error","Enter the time in the correct format")
                    startTimeEntry.delete(0, 'end')
            else:
                tkinter.messagebox.showinfo("Error","Enter the time in the correct format")
                startTimeEntry.delete(0, 'end')
        else:
            tkinter.messagebox.showinfo("Error","Enter the time in the correct format")
            startTimeEntry.delete(0, 'end')
    return status

def submit_data():
    genname = str(genNameEntry.get())
    startdate = str(startDateEntry.get())
    starttime = str(startTimeEntry.get())
    stopdate = str(stopDateEntry.get())
    stoptime = str(stopTimeEntry.get())
    loadvalue = str(loadValueEntry.get())
    runreason = str(runReasonEntry.get())

    
    if genname not in genNameList:
        tkinter.messagebox.showinfo("Generator name must be from the following fields", ', '.join(genNameList))
        genNameEntry.delete(0, 'end')
        startDateEntry.delete(0, 'end')
        startTimeEntry.delete(0, 'end')
        stopDateEntry.delete(0, 'end')
        stopTimeEntry.delete(0, 'end')
        loadValueEntry.delete(0, 'end')
        runReasonEntry.delete(0, 'end')
    else:
        statstarttime = check_time(starttime,startTimeEntry)
        statstoptime = check_time(stoptime,stopTimeEntry)

        # print('Stat Starttime',statstarttime)

        if(statstarttime and statstoptime):
            if (len(genname) == 0 or len(startdate) == 0 or len(starttime) == 0 or len(stopdate) == 0 or len(stoptime) == 0 or len(loadvalue) == 0 or len(runreason) == 0):
                tkinter.messagebox.showinfo("Missing Fields", "Some fields empty")
            else:
                firstresult = check_capacity(genname)
                tkinter.messagebox.showinfo("Generator Capacity", firstresult)
                # change the format of datetime
                try:
                    startdate = datetime.strptime(startdate, '%m/%d/%y')
                    startdate = datetime.strftime(startdate, '%Y-%m-%d')
                except:
                    pass
                try:
                    stopdate = datetime.strptime(stopdate, '%m/%d/%y')
                    stopdate = datetime.strftime(stopdate, '%Y-%m-%d')
                except:
                    pass
                
                
                secondresult = dump_data(genname,startdate,starttime,stopdate,stoptime,loadvalue,runreason)
                tkinter.messagebox.showinfo("Generator Result", secondresult)
        else:
            tkinter.messagebox.showinfo("Data Error", "Error while filling the fields")

def displaydata():
    genName = str(genNameEntry.get())
    startDate = str(startDateEntry.get())
    stopDate = str(stopDateEntry.get())
    if(genName not in genNameList):
        tkinter.messagebox.showinfo("Generator name must be from the following fields", ', '.join(genNameList))
        genNameEntry.delete(0, 'end')
    else:    
        genData = pd.read_excel('Gen_Data.xlsx', sheet_name='GEN '+genName, parse_dates=['StartDate', 'StopDate'])
        mask = (genData.StartDate >= startDate) & (genData.StartDate <= stopDate)


        dataDisplay = Tk()
        canvas = tk.Canvas(dataDisplay, width=820, height=500, relief='raised')
        scroll_y = tk.Scrollbar(dataDisplay, orient="vertical", command=canvas.yview)
        root = tk.Frame(canvas)
        root.title = ('Generator data')

        columnLabel = genData.columns.tolist()

        for h in range(0, len(columnLabel)):
            tk.Label(root, text='Index').grid(row = 0, column = 0)
            tk.Label(root, text=columnLabel[h]).grid(row = 0, column = h+1)

        for i, row in genData[mask].iterrows():
            print('i: ', i, '\nrow: ', row)
            row['StartDate'] = row['StartDate'].strftime('%Y-%m-%d')
            row['StopDate'] = row['StopDate'].strftime('%Y-%m-%d')
            tk.Label(root, text=genData.index[i]).grid(row=i, column=0)
            c = 0
            for cell in row:
                tk.Label(root, text=cell).grid(row=i, column=c+1)
                c += 1

        # put the frame in the canvas
        canvas.create_window(0, 0, anchor='nw', window=root)
        # make sure everything is displayed before configuring the scrollregion
        canvas.update_idletasks()

        canvas.configure(scrollregion=canvas.bbox('all'), 
                         yscrollcommand=scroll_y.set)

        canvas.pack(fill='both', expand=True, side='left')
        scroll_y.pack(fill='y', side='right')    

        root.mainloop()

def edit_window(prevWin, genName, index):
    close_prev(prevWin)
    newWindow2 = tk.Toplevel(loginWin)
    newWindow2.title("Edit Generator Data")
    canvas3 = Canvas(newWindow2, width=750, height=550, relief='raised')
    canvas3.pack()
    
    headerLabel = Label(newWindow2, text='Editing data for GEN ' + genName + ', index number '+str(index))
    headerLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 50, window=headerLabel)
    
    startDateLabel = Label(newWindow2,
                         text='Start date in format mm-dd-yy')
    startDateLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 80, window=startDateLabel)

    global startDateEntry
    global startDateEdit
    startDateEntry = DateEntry(newWindow2)
    canvas3.create_window(510, 80, window=startDateEntry)
    startDateEntry.delete(0, 'end')
    startDateEntry.insert(0, str(startDateEdit))
    
    startTimeLabel = Label(newWindow2,
                           text='Enter the start time in format hh:mm')
    startTimeLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 120, window=startTimeLabel)

    global startTimeEntry
    global startTimeEdit
    startTimeEntry = Entry(newWindow2)
    canvas3.create_window(510, 120, window=startTimeEntry)
    startTimeEntry.insert(0, str(startTimeEdit))
        
    stopDateLabel = Label(newWindow2,
                           text='Enter the stop date in format mm-dd-yy')
    stopDateLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 160, window=stopDateLabel)

    global stopDateEntry
    global stopDateEdit
    stopDateEntry = DateEntry(newWindow2)
    canvas3.create_window(510, 160, window=stopDateEntry)
    stopDateEntry.delete(0, 'end')
    stopDateEntry.insert(0, str(stopDateEdit))
    
    stopTimeLabel = Label(newWindow2,
                           text='Enter the stop time in format hh:mm')
    stopTimeLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 200, window=stopTimeLabel)

    global stopTimeEntry
    global stopTimeEdit
    stopTimeEntry = Entry(newWindow2)
    canvas3.create_window(510, 200, window=stopTimeEntry)
    stopTimeEntry.insert(0, str(stopTimeEdit))
    
    loadValueLabel = Label(newWindow2,
                          text='Load Value')
    loadValueLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 240, window=loadValueLabel)
    
    global loadValueEntry
    global loadValueEdit
    loadValueEntry = Entry(newWindow2)
    canvas3.create_window(510, 240, window=loadValueEntry)
    loadValueEntry.insert(0, str(loadValueEdit))
    
    runReasonLabel = Label(newWindow2,
                          text='Reason for Run')

    runReasonLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 280, window=runReasonLabel)

    global runReasonEntry
    global runReasonEdit
    runReasonEntry = Entry(newWindow2)
    canvas3.create_window(510, 280, window=runReasonEntry)
    runReasonEntry.insert(0, str(runReasonEdit))
    
    
    
    submitDataBtn = Button(newWindow2, text='Insert new data', bg='green', fg='white', command=lambda : insert_entry(genName, index, newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(440, 350, window=submitDataBtn)
    delDataBtn = Button(newWindow2, text='Delete this Entry', bg='green', fg='white', command=lambda : delete_entry(newWindow2, genName, index),
                          font=('helvetica', 14))
    canvas3.create_window(440, 400, window=delDataBtn)
    mainMenuBtn = Button(newWindow2, text='Main Menu', bg='green', fg='white', command=lambda : select_operation(newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(440, 450, window=mainMenuBtn)
    logoutBtn = Button(newWindow2,text='Logout', bg='green', fg='white', command=lambda :signout(newWindow2),
                              font=('helvetica', 14))
    canvas3.create_window(440, 500, window=logoutBtn)
    
def edit_window_mod():
    genname = str(genNameEntry.get())
    if(genname not in genNameList):
        tkinter.messagebox.showinfo("Generator name must be from the following fields", ', '.join(genNameList))
        genNameEntry.delete(0, 'end')
    else:
        startdate = str(startDateEntry.get())
        stopdate = str(stopDateEntry.get())
        genData = pd.read_excel('Gen_Data.xlsx', sheet_name='GEN ' + genname, parse_dates=['StartDate', 'StopDate'])
        print(genData)
        print('=========')
        mask = (genData.StartDate >= startdate) & (genData.StartDate <= stopdate)

        dataDisplay = Tk()
        canvas = tk.Canvas(dataDisplay, width=820, height=500, relief='raised')
        scroll_y = tk.Scrollbar(dataDisplay, orient="vertical", command=canvas.yview)
        root = tk.Frame(canvas)
        root.title = ('Generator data')

        columnLabel = ['Index']
        columnLabel = genData.columns.tolist()

        for h in range(0, len(columnLabel)):
            tk.Label(root, text=columnLabel[h]).grid(row=0, column=h+1)

        for i, row in genData[mask].iterrows():
            tk.Entry(root, text=genData.index[i]).grid(row=i, column=0)
            c = 1
            for cell in row:
                entry = tk.Entry(root,width=10)
                entry.insert(0, cell)
                entry.grid(row=i + 1, column=c)
                c += 1

        tk.Button(root, text='Edit the Data', bg='green', fg='white', command=lambda: edit_data(dataDisplay, genData, mask),
                  font=('helvetica', 14)).grid(row=10, column=3)

        # put the frame in the canvas
        canvas.create_window(0, 0, anchor='nw', window=root)
        # make sure everything is displayed before configuring the scrollregion
        canvas.update_idletasks()

        canvas.configure(scrollregion=canvas.bbox('all'),
                         yscrollcommand=scroll_y.set)

        canvas.pack(fill='both', expand=True, side='left')
        scroll_y.pack(fill='y', side='right')

        tk.Button(root, text='Main Menu', bg='green', fg='white', command=lambda: dataDisplay.destroy(),
                  font=('helvetica', 14)).grid(row=10, column=7)

        root.mainloop()

def edit_data_mod(leaveWin, genData, mask):
    proceed = yes_no(leaveWin)
    if proceed == True:
        for i, row in genData[mask].iterrows():
            c = 0
            for cell in row:
                entry = tk.Entry(leaveWin,width=10).grid(row=i + 1, column=c)
                print(entry.get())
        leaveWin.destroy()

        tkinter.messagebox.showinfo("Data Status", "Data has been updated")
        
def edit_data(prevWin):
    close_prev(prevWin)
    newWindow2 = tk.Toplevel(loginWin)
    newWindow2.title("Edit Generator Data")
    canvas3 = Canvas(newWindow2, width=820, height=250, relief='raised')
    canvas3.pack()

    genNameLabel = Label(newWindow2, text=('Please select the generator from the following:\n', ', '.join(genNameList)))
    genNameLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 50, window=genNameLabel)

    global genNameEntry
    genNameEntry = Entry(newWindow2)
    canvas3.create_window(510, 50, window=genNameEntry)

    indexLabel = Label(newWindow2, text='Please enter the index number you wish to edit:')
    indexLabel.config(font=('helvetica', 12))
    canvas3.create_window(250, 120, window=indexLabel)

    global indexEntry
    indexEntry = Entry(newWindow2)
    canvas3.create_window(510, 120, window=indexEntry)
    
    editQueryBtn = Button(newWindow2, text='Select', bg='green', fg='white', command=lambda : edit_query(newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(750, 120, window=editQueryBtn)
    mainMenuBtn = Button(newWindow2, text='Main Menu', bg='green', fg='white', command=lambda : select_operation(newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(440, 200, window=mainMenuBtn)

def edit_query(prevWin):
    genName = str(genNameEntry.get())
    index = str(indexEntry.get())
    if genName not in genNameList:
        tkinter.messagebox.showinfo("Generator name must be from the following fields", ', '.join(genNameList))
        genNameEntry.delete(0, 'end')
    elif index == '' or not index.isnumeric:
        tkinter.messagebox.showinfo('Invalid Entry',"Please select an index number to modify")
        indexEntry.delete(0, 'end')
    else:
        genData = pd.read_excel('Gen_Data.xlsx', sheet_name='GEN '+genName)
        index = int(index)
        if index not in genData.index:
            tkinter.messagebox.showinfo('Invalid Entry',"That index number does not exist")
            indexEntry.delete(0, 'end')
        else:
            global startDateEdit
            startDateEdit = genData.loc[index]['StartDate']

            global startTimeEdit
            startTimeEdit = genData.loc[index]['StartTime']

            global stopDateEdit
            stopDateEdit = genData.loc[index]['StopDate']

            global stopTimeEdit
            stopTimeEdit = genData.loc[index]['StopTime']

            global loadValueEdit
            loadValueEdit = genData.loc[index]['Load']

            global runReasonEdit
            runReasonEdit = genData.loc[index]['ReasonForRun']

            edit_window(prevWin, genName, index)

def delete_entry(prevWin, genName, index):
    proceed = yes_no(prevWin)
    if proceed == True:
        delete_data(genName, index)
        tkinter.messagebox.showinfo('Success', 'This entry has been deleted')
        view_window(prevWin)

def insert_entry(genName, index, prevWin):
    startdate = str(startDateEntry.get())
    starttime = str(startTimeEntry.get())
    stopdate = str(stopDateEntry.get())
    stoptime = str(stopTimeEntry.get())
    loadvalue = str(loadValueEntry.get())
    runreason = str(runReasonEntry.get())
    proceed = yes_no(prevWin)
    if proceed == True:
        # change the format of datetime
        try:
            startdate = datetime.strptime(startdate, '%m/%d/%y')
            startdate = datetime.strftime(startdate, '%Y-%m-%d')
        except:
            pass
        try:
            stopdate = datetime.strptime(stopdate, '%m/%d/%y')
            stopdate = datetime.strftime(stopdate, '%Y-%m-%d')
        except:
            pass
        
        edit_row(genName, index, startdate, starttime, stopdate, stoptime, loadvalue, runreason)
        tkinter.messagebox.showinfo('Success', 'This entry has been modified')
        view_window(prevWin)
        
def report_gen_win(prevWin):
    close_prev(prevWin)
    newWindow2 = tk.Toplevel(loginWin)
    newWindow2.title("Generator Data Calculation Year")
    canvas3 = Canvas(newWindow2, width=600, height=200, relief='raised')
    canvas3.pack()

    selectYear = Label(newWindow2,
                         text='Enter the year')
    selectYear.config(font=('helvetica', 12))
    canvas3.create_window(250, 50, window=selectYear)

    global yearentry
    yearentry = Entry(newWindow2)
    canvas3.create_window(390, 50, window=yearentry)

    submitDataBtn = Button(newWindow2, text='Create Report', bg='green', fg='white', command=lambda : create_report(newWindow2),
                           font=('helvetica', 14))
    canvas3.create_window(300, 100, window=submitDataBtn)
    mainMenuBtn = Button(newWindow2, text='Main Menu', bg='green', fg='white', command=lambda : select_operation(newWindow2),
                          font=('helvetica', 14))
    canvas3.create_window(300, 150, window=mainMenuBtn)

def create_report(prevWin):
    dataYear = str(yearentry.get())
    if (len(dataYear) == 0):
        tkinter.messagebox.showinfo("Missing Fields", "Please Fill in the credentials")
    elif (len(dataYear) != 4) or not dataYear.isnumeric():
        tkinter.messagebox.showinfo("Invalid format", "Please Enter a valid year i.e. 2020")
    else:
        try:
            strr = create_summary(dataYear)
            tkinter.messagebox.showinfo("Data Report", strr)
        except:
            tkinter.messagebox.showerror("Data Report", 'No data exists for '+ dataYear)
            close_prev(prevWin)
            report_gen_win(prevWin)

def closedriver():
    progressbar.stop()
    sys.exit("The player doesn't want to play again")

adminRights = False

loginWin=Tk()
loginWin.title("Project Prototype")
canvas1 = Canvas(loginWin, width=720, height=250, relief='raised')
canvas1.pack()

label1 = Label(loginWin, text='Login to the System')
label1.config(font=('helvetica', 17))
canvas1.create_window(310, 40, window=label1)

label2 = Label(loginWin, text='Enter the username:')
label2.config(font=('helvetica', 12))
canvas1.create_window(250, 90, window=label2)

userNameEntry = Entry(loginWin)
canvas1.create_window(470, 90, window=userNameEntry)

label4 = Label(loginWin, text='Enter password:')
label4.config(font=('helvetica', 12))
canvas1.create_window(250, 130, window=label4)

passwordEntry = Entry(loginWin, show="*")
canvas1.create_window(470, 130, window=passwordEntry)



loginbtn = Button(text='Sign in', bg='green', fg='white', command = login,
                           font=('helvetica', 14))
stat = True
if(stat):
    loginbtn.state=tk.DISABLED


canvas1.create_window(340, 180, window=loginbtn)

loginWin.mainloop()

ModuleNotFoundError: No module named 'tkcalendar'